In [ ]:
data1['DateTime'] = data1['Date'].astype(str) + " " + data1['Time'].astype(str)  # převod datového typu a sloučení sloupců

data1['DateTime'] =  pn.to_datetime(data1["DateTime"], format="%m/%d/%Y %H:%M:%S.%f") # převod na datetime formát

first_column = data1.pop('DateTime')     # přesun sloupce na první místo
data1.insert(0, 'DateTime', first_column)   

data1  = data1.drop(['Date', 'Time'], axis = 'columns')  # smazání původních sloupců datum + čas



In [ ]:
data1 = data1.ffill(axis = 'index')       #doplnění chybějícich hodnot
count1 = data1.isna().any(axis=1).sum()    #počet řádků s buňkami které nebyly doplněny
missingData1 = data1.head(count1)   #dataset ke smazání (prvních x řádků)
length_m1 = missingData1['DateTime'].iloc[-1] - missingData1['DateTime'].iloc[0]   #délka smazaných dat
length1 = data1['DateTime'].iloc[-1] - data1['DateTime'].iloc[0]  #délka celkových dat
data1_dropped = data1.dropna()   #dataset k dalšímu zpracování bez smazaných řádků
lengthd_1 = data1_dropped['DateTime'].iloc[-1] - data1_dropped['DateTime'].iloc[0]   #délka dále zpracováváných dat


In [ ]:
data1_dropped = data1_dropped.replace(
    ["On (1)", "Off (0)", "Disabled (1)", "Idle (2)", "Acquiring (3)", "Maintain (4)", "Heatingonrequest (5)","Shuttingdown (6)"], 
    ["1","0","1","2","3","4","5","6"])

# náhrada číslenými hodnotami

data1_dropped['ServerWatchdog'] = data1_dropped['ServerWatchdog'].astype(int)
data1_dropped['BeamIsOn'] = data1_dropped['BeamIsOn'].astype(int)
data1_dropped['ExternalState'] = data1_dropped['ExternalState'].astype(int)
data1_dropped['IsBlanked'] = data1_dropped['IsBlanked'].astype(int)
data1_dropped['ServiceModeIsOn'] = data1_dropped['ServiceModeIsOn'].astype(int)

# převod na integer

In [ ]:
df_ServiceMode1 = data1_dropped[['DateTime', 'ServiceModeIsOn']]  #výběr relevantních sloupečků
df_ServiceMode1 = df_ServiceMode1.loc[df_ServiceMode1['ServiceModeIsOn'].diff() != 0]  #výběr řádků kde se měnil stav
Intervals1 = df_ServiceMode1.diff() # rozdíl mezi řádky
InServiceMode1 = Intervals1.loc[Intervals1['ServiceModeIsOn'] == -1].sum() #výpočet doby kdy byl mikroskop v servisním módu/nefunkční
NotInServiceMode1 = Intervals1.loc[Intervals1['ServiceModeIsOn'] == 1].sum() #výpočet doby kdy byl mikroskop v provozu
InSM1 = (InServiceMode1[0].total_seconds()/3600)/lengthd_1.days #přepočet na průměr za den
NotInSM1 = (NotInServiceMode1[0].total_seconds()/3600)/lengthd_1.days



In [ ]:
data1_dropped['Extraction'] = np.where((data1_dropped['BeamIsOn'] == 1) &  #podmínka a sloupec Extraction s jejím (ne)splněním
                     (data1_dropped['ExternalState'] == 4) &
                     (data1_dropped['IsBlanked'] == 0) &
                     (data1_dropped['ServerWatchdog'] == 1) &
                     (data1_dropped['ServiceModeIsOn'] == 0), '1', '0')

data1_dropped['Extraction'] = data1_dropped[('Extraction')].astype(int)  #převod z bool na int
extraction1 = data1_dropped[['DateTime', 'Extraction']] #výběr relevantních sloupečků
extraction1 = extraction1.loc[extraction1['Extraction'].diff() != 0] #výběr řádků kde se měnil stav
df1_extraction = extraction1.diff() #rozdíl mezi řádky
Extraction_time1 = df1_extraction.loc[df1_extraction['Extraction'] == -1].sum()  #výpočet doby kdy byl mikroskop aktivně využíván
ET1 = (Extraction_time1[0].total_seconds()/3600) #přepočet na hodiny
ET1_avg = ET1/lengthd_1.days #přepočet na průměr za den


In [ ]:
df1_externalState = data1_dropped.drop(["ServerWatchdog", "BeamIsOn", "IsBlanked", "ServiceModeIsOn"], axis= 'columns') #dataset bez přebytečných sloupců
df1_externalState = df1_externalState.loc[df1_externalState['ExternalState'].diff() != 0] #výběr řádků se změnou
df1_externalState['Difference'] = df1_externalState['DateTime'].diff() #výpočet časových intervalů a jejich uložení do nového sloupce
df1_externalState['Difference'] = df1_externalState['Difference'].shift(-1)  #sloupec bylo nutné posunout
df1_groupped = df1_externalState.groupby(['ExternalState']).sum()  #seskupení řádků podle jednotlivých kategorií




In [ ]:
def minutes(x):
   return x.total_seconds()/3600

df1_groupped['Difference'] = df1_groupped['Difference'].apply(minutes) #funkce k přepočtu na hodiny a příklad jejího použití

df1_groupped['Difference'] = (df1_groupped['Difference']/df1_groupped['Difference'].sum())*100  #přepočet na procenta